# Exporting FasterRCNN from Pytorch to ONNX format. 
<br>
FasterRCNN has based on different types. In this case we are exporting fasterRCNN model with compination of ResNet50 backbone and the FPN a neural network architecture designed to handle objects at different scale, which create a multi-scale feature pyramid from a single-scale input image.

## Loading the pre-trained model

In [ ]:
import torch 
from torchvision.models.detection import fasterrcnn_resnet50_fpn


model = fasterrcnn_resnet50_fpn(pretrained=True)

#### creating a dummy input and adjusting the size according to the model specifications
<br>
FasterRCNN batched in (B, C, H, W) 

In [ ]:
input = torch.randn(1, 3, 800, 800)

There are two types senarios with dummy input in this case. According to pytorch uses `x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]`

In [ ]:
#x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]

Exporting FasterRCNN to ONNX needs different procedure. This is happening because of object detection tasks. The input in object detection has usually a batch of images and each tensor in the x list coresponds to one image image in the batch. In this case using images of different sizes, the model can handle more variations of input sizes. 


Summary:
<br>
The dummy input in the model represents two different-sized images to demonstrate the model's ability to handle variable input dimensions. It is for simplicity reasons to use a standard input tensor rather than a variety of input tensors. 

In [ ]:
""" torch.onnx.export(
    model,
    x,
    "fasterrcnn_resnet50_fpn.onnx",
    input_names=["input"],
    output_names=["output"]
) """

## Export the model to ONNX format

In [ ]:
torch.onnx.export(model,
                  input,
                  "fasterrcnn_resnet50_fpn.onnx",
                  input_names=["input"],
                  output_names=["output"],
                  dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

Dynamic axes specifies that the input can vary in dimencions, more or less than 800x800px while output has in this case fixed output dims. Sometimes output dimensions can also vary depending on the architecture of the given model. In this case it is assigned using more batched data than in given input with variable width and height input featured map `dynamic_axes={"input": {0: "batch_size", 2: "height", 3: "width"}, "output": {0: "batch_size"}}`. 

## Verify ONNX Model

In [43]:
import onnxruntime as ort 

fasterrcnn_resnet50_fpn_onnx = ort.InferenceSession("fasterrcnn_resnet50_fpn.onnx")

print("Input names:", fasterrcnn_resnet50_fpn_onnx.get_inputs()[0].name)
print("Output names:", fasterrcnn_resnet50_fpn_onnx.get_outputs()[0].name)

Input names: input
Output names: output


# Verification of the Exported Model Using Netron

<img src="input_fasterrcnn.png" alt="Input of fasterrcnn in Netron" width="1000" height="500">


<img src="output_fasterrcnn.png" alt="Ouput of fasterrcnn in Netron" width="1000" height="500">